In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import sys,os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils import clean_df

load_dotenv(dotenv_path="../.env")
file_path = os.getenv("FILEPATH2")

df = pd.read_csv(file_path)

In [2]:
df_cleaned = clean_df(df)

# Creating Month column 
df_cleaned['Transaction Date'] = pd.to_datetime(df_cleaned['Transaction Date'])
df_cleaned['Month'] = df_cleaned['Transaction Date'].dt.to_period('M')

df_cleaned2 = df_cleaned.copy()
df_cleaned.head(3)

,Transaction Date,Post Date,Description,Category,Type,Amount,Was Venmoed,Split Count,Month
0,2025-10-29,10/30/2025,FSPROYAL OAK GOLF CEN,Health & Wellness,Sale,14.50,,1,2025-10
1,2025-10-21,10/22/2025,DTE ENERGY,Bills & Utilities,Sale,113.46,,1,2025-10
5,2025-10-05,10/6/2025,Spotify USA,Bills & Utilities,Sale,11.99,,1,2025-10


In [3]:
# 1) Recurring subscriptions (merchant appears >= n months with similar amounts)
months_seen = 6

monthly_counts = df_cleaned.groupby(['Description', 'Month'])['Amount'].sum().reset_index()
recurrence = monthly_counts.groupby('Description').size().reset_index(name='months_seen')
recurring_merchants = recurrence[recurrence['months_seen'] >= months_seen]['Description'].tolist()
df_cleaned['is_recurring_candidate'] = df_cleaned['Description'].isin(recurring_merchants)
df_cleaned[df_cleaned['is_recurring_candidate'] == True]['Description'].value_counts()

# Groceries (costco), gas for car (costco), Insurance, Golf, DTE Electric, Spotify, Chipotle

Description
COSTCO WHSE                 36
COSTCO GAS                  34
MEIJER STORE                26
CHIPOTLE                    23
FSPROYAL OAK GOLF CEN       20
AAA INSURANCE GW EFT        20
FSPROYAL OAK GOLF CENTER    18
UBER TRIP                   16
SQ OASIS GOLF PRACTIC       12
Spotify USA                 10
KROGER                      10
DTE ENERGY                  10
VISIBLE                      9
Name: count, dtype: int64

In [4]:
# 2) Frequent small purchases (many transactions with small avg amount)
freq = 5
mean_dollars = 25
merchant_stats = df_cleaned.groupby('Description')['Amount'].agg(['count', 'mean']).reset_index()
freq_small = merchant_stats[(merchant_stats['count'] >= freq) & (merchant_stats['mean'] < mean_dollars)]
df_cleaned['is_freq_small'] = df_cleaned['Description'].isin(freq_small['Description'])
df_cleaned[df_cleaned['is_freq_small'] == True]['Description'].value_counts()

# Driving Range, NYCT (I think this is NY subway), Spotify, Chipotle, Car Wash

Description
CHIPOTLE                    23
FSPROYAL OAK GOLF CEN       20
FSPROYAL OAK GOLF CENTER    18
MTANYCT PAYGO               11
Spotify USA                 10
GLFGOLFNOWRESERVATIO         7
TARGET                       6
LITTLE CAESARS               5
ELEVEN                       5
FARMINGTON HILLS GOLF        5
VORTEX BRIGTHON              5
Name: count, dtype: int64

In [5]:
# 3) One-off large purchases relative to overall spending behavior (global z-score)
global_mean = df_cleaned['Amount'].mean()
global_std = df_cleaned['Amount'].std()
print(global_mean)
print(global_std)

df_cleaned['z_score_global'] = (df_cleaned['Amount'] - global_mean) / global_std
df_cleaned['is_one_off_large'] = df_cleaned['z_score_global'] > 3
df_cleaned[df_cleaned['is_one_off_large'] == True]['Description'].value_counts()

# Vegas hotel, new glasses, football tickets for 6 people, golfing in alaska for 6, whitewater rafting alaska for 6
# Flight to alaska, another flight to somewhere, passport (Was paid back for)

45.99523188405797
73.01216983486725


Description
DELTA AIR                   2
WARBY PARKER                1
MGM GRAND FRONT DESK        1
SEATGEEK TICKETS            1
FSPANCHORAGE GOLF COURSE    1
SQ NOVA RIVER RUNNERS IN    1
SQ INTERNATIONAL EXPRESS    1
Name: count, dtype: int64

In [6]:
df_cleaned['z_score_global'] = (df_cleaned['Amount'] - global_mean) / global_std
df_cleaned['is_one_off_large2'] = df_cleaned['z_score_global'] > 2
df_cleaned[df_cleaned['is_one_off_large2'] == True]['Description'].value_counts()

Description
DTE ENERGY                  2
SEATGEEK TICKETS            2
DELTA AIR                   2
MGM GRAND FRONT DESK        1
GLFRACKHAMGC                1
RPSTwin Arbors CD           1
OMNIA NIGHTCLUB             1
SLOTZILLA BOX OFFICE        1
WARBY PARKER                1
MGM GRAND ADV DEP           1
FSPANCHORAGE GOLF COURSE    1
COSTCO WHSE                 1
SQ NOVA RIVER RUNNERS IN    1
SQ INTERNATIONAL EXPRESS    1
Name: count, dtype: int64

In [7]:
# 5) New merchant (first seen in last 90 days)
first_seen = df_cleaned.groupby('Description')['Transaction Date'].min().reset_index(name='first_seen')
df_cleaned = df_cleaned.merge(first_seen, on='Description', how='left')
df_cleaned['days_since_first'] = (pd.Timestamp.today() - pd.to_datetime(df_cleaned['first_seen'])).dt.days
df_cleaned['is_new_merchant'] = df_cleaned['days_since_first'] <= 60
df_cleaned[df_cleaned['is_new_merchant'] == True]['Description'].value_counts()

# Restuarants, drinks at bar and other dumb stuff 

Description
FSPFAULKWOOD SHORES G      4
OTOOLES                    3
UNIVERSITY OF MICHIGAN     3
Lion Kenilworth            2
SQ ESJ GOLF LLC DBA SAL    2
DAVES HOT CHICKEN          2
WAITROSE                   2
CTLPVARIETY FOODSERVI      2
OTOOLES SHOT GIRL          2
TIN ROOF DETROIT LLC       2
TSTLITTLE GHOST            1
CULVERS OF HOWELL          1
RPSTwin Arbors CD          1
GLFCHANDLERPARKGC          1
AmazoncomJAYP              1
SQ APOTHECARY COFFEE       1
SQ HUDDLE SOFT SERVE       1
UNIV OF MICHIGAN FLEX      1
VEND LLC                   1
WENDYS SOUTHFIELD          1
BELLE TIRE                 1
THE HOME DEPOT             1
MILE amp HAGGERTY          1
TST BUDDYS PIZZA DEAR      1
MARATHON                   1
CANTINA LAREDO             1
WH SMITH                   1
SHAKE SHACK                1
TSTGRAND RIVER BREWERY     1
TIMENEWSST                 1
Shop Dutch discoveries     1
EXXON MICHIGAN FUELS M     1
DTW ChickfilA              1
Name: count, dtype: int64

In [11]:
flags = [
    'is_recurring_candidate',
    'is_freq_small',
    'is_one_off_large2',
    'is_new_merchant'
]

df_cleaned['cut_score'] = df_cleaned[flags].sum(axis=1)
candidate_df = df_cleaned[df_cleaned['cut_score'] >= 2].sort_values('cut_score', ascending=False)
candidate_df['Description'].value_counts()
print(candidate_df.groupby('Description')['Amount'].sum())

# Driving Range and Chipotle

## Estimate of Savings
print(f"Total Potential Savings: ${candidate_df['Amount'].sum()}")

Description
CHIPOTLE                    297.28
COSTCO WHSE                 203.10
DTE ENERGY                  450.51
FSPROYAL OAK GOLF CEN       294.00
FSPROYAL OAK GOLF CENTER    257.25
RPSTwin Arbors CD           250.00
Spotify USA                 119.90
Name: Amount, dtype: float64
Total Potential Savings: $1872.0400000000002


In [ ]:
# # "Group transactions by $0–100, $100–200, etc."
df_cleaned2['Amount_Bin'] = pd.cut(df_cleaned2['Amount'], bins=10)

bin_summary = (
    df_cleaned2.groupby('Amount_Bin')
    .agg(
        mean_spent=('Amount', 'mean'),
        total_spent=('Amount', 'sum'),
        n_transactions=('Amount', 'count'),
        unique_merchants=('Description', 'nunique')
    )
    .reset_index()
)

bin_summary

C:\Users\matt\AppData\Local\Temp\ipykernel_37452\657410384.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_cleaned2.groupby('Amount_Bin')


,Amount_Bin,mean_spent,total_spent,n_transactions,unique_merchants
0,"(0.165, 77.427]",23.832716,13775.31,578,240
1,"(77.427, 153.924]",111.466456,8805.85,79,41
2,"(153.924, 230.421]",178.424091,3925.33,22,13
3,"(230.421, 306.918]",261.087500,1044.35,4,4
4,"(306.918, 383.415]",348.960000,348.96,1,1
5,"(383.415, 459.912]",NaN,0.00,0,0
6,"(459.912, 536.409]",502.300000,502.30,1,1
7,"(536.409, 612.906]",578.000000,578.00,1,1
8,"(612.906, 689.403]",663.570000,1990.71,3,3
9,"(689.403, 765.9]",765.900000,765.90,1,1


In [ ]:
# "Group the lowest 10%, next 10%, ..., top 10% of transactions"
df_cleaned2['Amount_Bin'] = pd.qcut(df_cleaned2['Amount'], q=10)

bin_summary = (
    df_cleaned2.groupby('Amount_Bin')
    .agg(
        mean_spent=('Amount', 'mean'),
        total_spent=('Amount', 'sum'),
        n_transactions=('Amount', 'count'),
        unique_merchants=('Description', 'nunique')
    )
    .reset_index()
)

bin_summary

C:\Users\matt\AppData\Local\Temp\ipykernel_37452\2690635348.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_cleaned2.groupby('Amount_Bin')


,Amount_Bin,mean_spent,total_spent,n_transactions,unique_merchants
0,"(0.929, 5.988]",3.651739,251.97,69,36
1,"(5.988, 10.598]",8.769130,605.07,69,53
2,"(10.598, 13.697]",12.190870,841.17,69,39
3,"(13.697, 16.636]",14.741594,1017.17,69,30
4,"(16.636, 23.5]",19.373913,1336.80,69,51
5,"(23.5, 31.804]",27.922899,1926.68,69,37
6,"(31.804, 40.314]",34.935797,2410.57,69,36
7,"(40.314, 62.604]",52.318986,3610.01,69,45
8,"(62.604, 111.131]",83.928116,5791.04,69,42
9,"(111.131, 765.9]",202.119275,13946.23,69,33
